In [1]:
import pandas as pd
import re

In [2]:
import os
import pickle

folder_path = "../data/data_simbis/"

# Get a list of all files in the folder
pickle_files = [file for file in os.listdir(folder_path) if file.endswith('.pkl')]

# Loop through the list of pickle files and load them
loaded_pickles = []
for file_name in pickle_files:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'rb') as file:
        loaded_pickle = pickle.load(file)
        loaded_pickles.append(loaded_pickle)

# Now, loaded_pickles contains all the pickles from the folder
sim_df = pd.concat(loaded_pickles, ignore_index=True)

In [3]:
sim_df.dtypes

Iteration     object
Rewards       object
Prices        object
Epsilon       object
beta         float64
index          int64
dtype: object

In [17]:
sim_df['Rewards'][0][1]

0.28044202997287715

In [4]:
def extract_numbers(text):
    numbers = re.findall(r'-?\d+\.?\d*(?:e[-+]?\d+)?', text)
    return [float(num) for num in numbers]

# # Apply the function to the column and convert the strings into lists of two numbers
# sim_df['Rewards'] = sim_df['Rewards'].apply(extract_numbers)
# sim_df['Prices'] = sim_df['Prices'].apply(extract_numbers)
# sim_df['Epsilon'] = sim_df['Epsilon'].apply(extract_numbers)

#Convert into int if digit
sim_df['Iteration'] = sim_df['Iteration'].apply(lambda x: int(x) if str(x).isdigit() else x)


#Separate the profits, prices and Epsilon of each player
sim_df[['Reward1', 'Reward2']] = sim_df['Rewards'].apply(lambda x: pd.Series(x))
sim_df[['Price1', 'Price2']] = sim_df['Prices'].apply(lambda x: pd.Series(x))
sim_df[['Epsilon1', 'Epsilon2']] = sim_df['Epsilon'].apply(lambda x: pd.Series(x))

#Drop the useless columns
sim_df = sim_df.drop(columns=['Rewards', 'Prices', 'Epsilon'])

/var/folders/1v/lw80nwsd61d7sp0q_pzj0ppm0000gp/T/ipykernel_873/355113774.py:15: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  sim_df[['Reward1', 'Reward2']] = sim_df['Rewards'].apply(lambda x: pd.Series(x))
/var/folders/1v/lw80nwsd61d7sp0q_pzj0ppm0000gp/T/ipykernel_873/355113774.py:16: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  sim_df[['Price1', 'Price2']] = sim_df['Prices'].apply(lambda x: pd.Series(x))
/var/folders/1v/lw80nwsd61d7sp0q_pzj0ppm0000gp/T/ipykernel_873/355113774.py:17: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  sim_df[['Epsilon1', 'Epsilon2']] = sim_df['Epsilon'].apply(lambda x: pd.Series(x))


In [5]:
sim_df

,Iteration,beta,index,Reward1,Reward2,Price1,Price2,Epsilon1,Epsilon2
0,0,0.002500,1,0.200833,0.280442,1.6195,1.4635,1.000000e+00,1.000000e+00
1,10000,0.002500,1,0.217067,0.202776,1.4245,1.4635,1.392271e-11,9.048465e-01
2,20000,0.002500,1,0.298722,0.271664,1.6195,1.6585,1.933578e-22,8.187389e-01
3,30000,0.002500,1,0.207851,0.384671,1.8535,1.6195,2.685342e-33,7.408256e-01
4,40000,0.002500,1,0.212486,0.399740,1.8925,1.6585,3.729388e-44,6.703267e-01
...,...,...,...,...,...,...,...,...,...
190375,9970000,0.000025,10,0.159728,0.405275,1.9315,1.5805,5.650817e-109,5.021628e-44
190376,9980000,0.000025,10,0.159728,0.405275,1.9315,1.5805,4.400861e-109,4.543756e-44
190377,9990000,0.000025,10,0.159728,0.405275,1.9315,1.5805,3.427394e-109,4.111361e-44
190378,10000000,0.000025,10,0.159728,0.405275,1.9315,1.5805,2.669257e-109,3.720113e-44


In [20]:
sim_df.to_csv('../final_data/res_diff_epsilon.csv', index=False)

In [7]:
import os
import pickle

folder_path = "../data/data/data_Q/"

# Get a list of all files in the folder
pickle_files = [file for file in os.listdir(folder_path) if file.endswith('.pkl')]

# Loop through the list of pickle files and load them
loaded_pickles = []
for file_name in pickle_files:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'rb') as file:
        loaded_pickle = pickle.load(file)
        loaded_pickles.append(loaded_pickle)

# Now, loaded_pickles contains all the pickles from the folder
Q_df = pd.concat(loaded_pickles, ignore_index=True)

In [8]:
Q_df.to_csv('../final_data/Q_q2.csv', index=False)